# Занятие 08. 21.03.2024 От оценки параметров к проверке гипотез: создание байесовских А/В-тестов

## Концепт-карты

### Концепт-карта Глава 15

![Рисунок 1. Концепт-карта Глава 15](./Картинки/Главы/15.png)

### Концепт-карта Глава 16

![Рисунок 2. Концепт-карта Глава 16](./Картинки/Главы/16.png)

## Задача 1

### Условие задачи

Предположим, опытный директор по маркетингу говорит вам о своей
уверенности в том, что вариант без картинок (B) не будет работать
иначе, чем исходный вариант. Как это объяснить в нашей модели?
Внедрите это изменение и посмотрите, как изменятся окончательные
выводы

### Решение задачи

Вы можете объяснить это, увеличив силу априорной вероятности.

Например:

```r
prior.alpha <- 300
prior.beta <- 700
```

Это потребует гораздо больше доказательств для изменения убеждений.
Чтобы увидеть, как это меняет выводы, мы можем перезапустить код:

```r
a.samples <- rbeta(n.trials,36+prior.alpha,114+prior.beta)
b.samples <- rbeta(n.trials,50+prior.alpha,100+prior.beta)
p.b_superior <- sum(b.samples > a.samples)/n.trials
```

Тогда новое значение p.b_superior равно 0,74, что намного ниже, чем исходное значение 0,96.

### Ответ


In [ ]:
import numpy as np

n_trials = 1000
prior_alpha = 1
prior_beta = 1

a_samples = np.random.beta(36 + prior_alpha, 114 + prior_beta, n_trials)
b_samples = np.random.beta(50 + prior_alpha, 100 + prior_beta, n_trials)
p_b_superior = np.sum(b_samples > a_samples) / n_trials

p_b_superior

### Проверка

Нету

## Задача 2

### Условие задачи

Ведущий дизайнер видит ваши результаты и настаивает на том, что
вариант B без картинок не будет работать лучше. Она считает, что вы
должны принять коэффициент конверсии для варианта B, близкий
к 20 %, а не к 30 %. Реализуйте решение для этого и снова просмотрите
результаты анализа. 

### Решение задачи

Вместо того чтобы использовать одну априорную вероятность для изменения убеждений, стоит использовать две — одну, отражающую исходную априорную вероятность, для A, и другую, отражающую веру ведущего дизайнера, для B. Вместо того чтобы использовать слабую априорную вероятность, мы будем использовать немного более сильную:

```r
a.prior.alpha <- 30
a.prior.beta <- 70
b.prior.alpha <- 20
b.prior.beta <- 80
```

При запуске этого моделирования нужно использовать два отдельных
априорных значения:

```r
a.samples <- rbeta(n.trials,36+a.prior.alpha,114+a.prior.beta)
b.samples <- rbeta(n.trials,50+b.prior.alpha,100+b.prior.beta)
p.b_superior <- sum(b.samples > a.samples)/n.trials
```

На этот раз p.b_superior составляет 0,66, что ниже, чем раньше, но это все же
дает небольшую уверенность в том, что B может быть лучшим вариантом.

### Ответ


In [ ]:
import numpy as np

a_prior_alpha = 30
a_prior_beta = 70
b_prior_alpha = 20
b_prior_beta = 80

n_trials = 1000

a_samples = np.random.beta(36 + a_prior_alpha, 114 + a_prior_beta, n_trials)
b_samples = np.random.beta(50 + b_prior_alpha, 100 + b_prior_beta, n_trials)

p_b_superior = np.sum(b_samples > a_samples) / n_trials

p_b_superior



### Проверка

Нету

## Задача 3

### Условие задачи

Предположим, что 95 %-ная уверенность означает, что вы более или
менее «убеждены» в правильности гипотезы. Также предположим, что
больше нет ограничений на количество писем, которые можно отправить
в тесте. Если истинное преобразование для A составляет 0,25, а для B —
0,3, изучите, сколько выборок потребуется, чтобы убедить директора по
маркетингу в том, что B на самом деле лучше. Изучите то же самое для
ведущего дизайнера. Можно сгенерировать образцы конверсий с помощью следующего фрагмента R:

```r
true.rate <- 0.25
number.of.samples <- 100
results <- runif(number.of.samples) <= true.rate
```

### Решение задачи

Вот основной код для решения этой задачи в случае директора по маркетингу (для ведущего дизайнера вам нужно добавить отдельные априорные
вероятности). Можно использовать цикл while в R для перебора примеров
(или просто вручную пробовать новые значения).

```r
a.true.rate <- 0.25
b.true.rate <- 0.3
prior.alpha <- 300
prior.beta <- 700
number.of.samples <- 0
#using this as an initial value so that the loop starts
p.b_superior <- -1
while(p.b_superior < 0.95){
    number.of.samples <- number.of.samples + 100
    a.results <- runif(number.of.samples/2) <= a.true.rate
    b.results <- runif(number.of.samples/2) <= b.true.rate
    a.samples <- rbeta(n.trials,
    sum(a.results==TRUE)+prior.alpha,
    sum(a.results==FALSE)+prior.beta)
    b.samples <- rbeta(n.trials,
    sum(b.results==TRUE)+prior.alpha,
    sum(b.results==FALSE)+prior.beta)
    p.b_superior <- sum(b.samples > a.samples)/n.trials
}
```

Обратите внимание, что поскольку этот код сам по себе является моделированием, вы будете получать разные результаты при каждом запуске,
поэтому запустите его несколько раз (или создайте более сложный пример,
который запускается еще несколько раз).

Чтобы убедить директора по маркетингу, нужно около 1200 образцов. Ведущему дизайнеру должно хватить около 1000 образцов. Обратите внимание,
что хотя ведущий дизайнер считает, что B хуже, в нашем примере он имеет
также более слабые априорные вероятности, поэтому требуется меньше
доказательств, чтобы изменить его мнение.

### Ответ


In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Данные для A и B
count_A = 100 # Предположим, что у нас есть 100 наблюдений для A
count_B = 100 # И 100 наблюдений для B

# Выполняем z-тест
stat, pval = proportions_ztest([count_A, count_B], [count_A + count_B, count_A + count_B], alternative='two-sided')

print(f"Z-статистика: {stat}, p-значение: {pval}")




### Проверка

Нету

## Задача 4

### Условие задачи

Возвращаясь к задаче с игральными костями, предположим, что ваш
друг допустил ошибку и внезапно осознал, что на самом деле было две
нечестные кости и только одна честная. Как это изменит априорный и,
следовательно, апостериорный шансы этой задачи? Вы более склонны
верить, что бросаемая кость нечестная?

### Решение задачи

Первоначальные априорные шансы были следующими:

$$
\frac{
    \frac{1}{3}
    }{
    \frac{2}{3}
} = \frac{1}{2}
$$

И коэффициент Байеса составил 3,77, что дало нам апостериорные шансы
со значением 1,89. Наши новые априорные шансы следующие:

$$
\frac{
    \frac{2}{3}
}{
    \frac{1}{3}
} = 2
$$

Итак, апостериорные шансы составляют 2 × 3,77 = 7,54. Сейчас мы определенно более склонны верить, что бросаемая кость нечестная, но апостериорные шансы все еще не очень высоки. Мы бы хотели собрать больше
доказательств, прежде чем полностью сдаться

### Ответ

апостериорные шансы составляю 7,54

### Проверка

нету

## Задача 5

### Условие задачи

Вернемся к примеру с редкими заболеваниями. Предположим, вы обратились к врачу и после чистки ушей заметили, что симптомы не исчезли.
Еще хуже, появился новый симптом: головокружение. Врач предлагает
другое возможное объяснение, лабиринтит — вирусную инфекцию внутреннего уха, при которой в 98 % случаев возникает головокружение.
Однако потеря слуха и шум в ушах менее распространены при этом
заболевании; потеря слуха происходит только в 30 % случаев, а шум
в ушах — только в 28 %. Головокружение также является возможным
симптомом вестибулярной шванномы, но встречается только в 49 % случаев. В общей численности населения 35 человек на миллион заболевают
лабиринтитом ежегодно. Каковы апостериорные шансы гипотезы, что
у вас лабиринтит, по сравнению с гипотезой о вестибулярной шванноме?

### Решение задачи

Мы немного запутаем ситуацию и сделаем H1 вероятностью лабиринтита, а H2 — вероятностью вестибулярной шванномы, поскольку мы уже
видели, насколько маловероятна вестибулярная шваннома. Необходимо
пересчитать каждую часть апостериорных шансов, потому что мы рассматриваем новую часть данных, «имеет головокружение», а также совершенно
новую гипотезу.
Начнем с коэффициента Байеса. Для H1 мы имеем:

P(D|H1) = 0,98 × 0,30 × 0,28 = 0,082.

Новая вероятность для H2:

P(D|H2) = 0,63 × 0,55 × 0,49 = 0,170.

Таким образом, коэффициент Байеса для новой гипотезы:

$$
\frac{
    P(D|H_1)
}{
    P(D|H_2)
} = 0.48
$$

Это означает, что, учитывая только коэффициент Байеса, наличие вестибулярной шванномы является примерно в два раза лучшим объяснением,
чем лабиринтит. Теперь нужно посмотреть на соотношение шансов:

$$
O(H_1) = \frac{
    P(D|H_1)
}{
    P(D|H_2)
} = \frac{
    \frac{35}{1000000}
} {
    \frac{11}{1000000}
} = 3,18
$$

Лабиринтит встречается гораздо реже, чем избыток ушной серы, и лишь
примерно в три раза чаще, чем вестибулярная шваннома. При сложении
апостериорных шансов мы можем увидеть:

$$
O(H_1) \cdot \frac{
    P(D|H_1)
}{
    P(D|H_2)
} = 3,18 \cdot 0,48 = 1,53
$$

Конечным результатом является то, что лабиринтит — лишь немного лучшее объяснение, чем вестибулярная шваннома.

### Ответ

лабиринтит — лишь немного лучшее объяснение, чем вестибулярная шваннома

### Проверка

Нету
